# Python Porgramm frei stehender Bogen

Der Code wird im Folgenden erst beschrieben und unter der jeweiligen Beschreibung eingebunden.

## Bestimmung der Funktionsgleichung

Hier werden die bnötigten Python Bibliotheken importiert.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from stl import mesh

-------------------

Beim Bau des Bogens, hab ich mir bestimmte Werte festgelegt.<br> 
Einerseits die Länge $L$ des Bogens (Bogenlänge zwischen den Nullstellen) und die Nullstelle x0, bzw. Hälfte der Bogenbreite (siehe Abbildung). Zu dem die Anzahl an Klötzen, die Breite eines Klotzes und die durch einen Klotz beschriebene Bogenlänge.<br>
Der Bogen wird durch eine hängende Kette beschrieben.

![Alt-Text](bilder/geogebra_x0_l.png)

In [11]:
L = 136
x0 = 30
anzahlKloetze = 17
bogenlaengeDurchKloetz = 8 #die Bogenlänge, die durch einen Klotz simuliert wird
breiteKloetze = 4

-------------------

Für die Funktion, welche die Kette beschreibt, gilt: $f(x)= a \cdot \cosh \left( \frac{1}{a}x \right) + y_{0}$ (siehe PDF !!!!)<br>
Um den Faktor a zu bestimmen wird folgender Ausdruck verwendet: $0 = L - 2a \cdot \sinh \left( \frac{1}{a} x \right)$ (siehe PDF!!!)<br>
Die numerische Bestimmung **der Nullstelle** erfolgt hier mit Hilfe von Intervallhalbierung:<br>
Beim Funktionsaufruf, wird eine untere und eine obere Grenze festgelegt. Für $a$ wird die Mitte zwischen diesen Werten gewählt.<br>
Danach wird in f_a $L - 2a \cdot \sinh \left( \frac{1}{a} x \right)$ bestimmt.<br>
Ist dieser Wert kleiner als $\frac{1}{10000000}$, also näherungsweise null, wird der Wert von a zurückgegeben.<br>
Anderen Falls wird geprüft ob der Wert von f_a größer oder kleier ist als Null. Ist f_a kleiner als null, ruft sich die Funktion rekursiv auf. Jedoch wird a als untere Grenze übergeben (oberGrenze bleibt die obere Grenze).<br>
Ist f_a größer als null, passiert das gleich, nur wird a als obere Grenze übergeben und die untere Grenze bleibt gleich.

In [12]:
def a_bestimmen(unterGrenze,oberGrenze):
    a = (unterGrenze+oberGrenze)/2
    f_a = (L-2*a*math.sinh(1/a*x0))
    if( abs(f_a) < 1/10000000 ):
        return a
    elif (f_a < 0):
            return a_bestimmen(a,oberGrenze);
    else:
        return a_bestimmen(unterGrenze,a);

In [13]:
a = a_bestimmen(10,16)
print("a : ", a)

a :  12.541937507689


---------

$y_{0} = -a \cdot \cosh \left( \frac{1}{a}x \right)$: (umgeformt!! siehe PDF!!! Fehlt noch in PDF)

In [14]:
y0 = -a*math.cosh(1/a*x0)

-------------------

Jetzt sind alle Größe der Funktion $f(x)= a \cdot \cosh \left( \frac{1}{a}x \right) + y_{0}$ bekannt:

In [15]:
kettenlinie = lambda x: (a*np.cosh(1/a * x)+y0)
ableitung = lambda x: (np.sinh(1/a * x)) #siehe Ableitungsregeln PDF

## Bestimmung der x und y-Werte

Die folgende Funktion gibt einen den x Wert bei gegebener Bogenlänge l zurück. (siehe PDF!!!)

In [16]:
def x_werte_l(l):
    return (np.arcsinh(l/a) * a)

-------------------

Jetzt wird eine Liste der x-Werte, welche die Klötze beschreiben, erzeugt. (siehe PDF!!!!)<br>
Dafür läuft eine for Schleife von -(der Hälfte der vorgegebenen Bogenlänge), hier die Nullstelle, da Faktor a und y0 so bestimmt wurden, dass der Bogen durch den Teil unterhalb der x-Achse beschrieben wird, bis zur +(Hälfte der vorgegeben Bogenlänge).<br>
In jedem Schleifendurchlauf, wird i um die größe der Variable laengeHolzStuecke (Länge vom Bogen, welche durch einen Klotz beschrieben wird) erhöht. Jetzt wird die Funktion x_werte_l(i) mit i als Parameter aufgerufen und die Rückgabe wird in die Liste geschrieben.

In [18]:
def x_werte_holzstuecke():
    x_werte_holzstuecke = []
    for i in range (-int(L/2), int(L/2+1), bogenlaengeDurchKloetz):
        x_werte_holzstuecke.append(x_werte_l(i))
    return x_werte_holzstuecke

In [19]:
x_werte = x_werte_holzstuecke()
print("x Werte:")
print(x_werte)

x Werte:
[-30.000000004186834, -28.459695195406937, -26.708667641693076, -24.682268897417455, -22.282466596571965, -19.352603653310144, -15.629065100279782, -10.666755857440647, -3.9351171525557147, 3.9351171525557147, 10.666755857440647, 15.629065100279782, 19.352603653310144, 22.282466596571965, 24.682268897417455, 26.708667641693076, 28.459695195406937, 30.000000004186834]


-------------------

Um eine Liste mit den zugehörigen y-Werte zu bekommen, läuft die folgende Funktion durch die Liste der x-Werte, welche als Parameter übergeben wird, und fügt den jeweiligen Funktionswert von kettenlinie = lambda x: (a*np.cosh(1/a * x)+y0) zur Liste hinzu.

In [20]:
def y_werte_holzstuecke(xWerteListe):
    yWerte = []
    for i in xWerteListe:
        yWerte.append(kettenlinie(i))
    return yWerte

In [21]:
y_werte = y_werte_holzstuecke(x_werte)
print("y Werte:")
print(y_werte)

y Werte:
[2.2700234580952383e-08, -7.8501259341093075, -15.655823782564553, -23.39434867569104, -31.024771774511088, -38.466330404901925, -45.53973992575219, -51.78894544268181, -55.982593128987965, -55.982593128987965, -51.78894544268181, -45.53973992575219, -38.466330404901925, -31.024771774511088, -23.39434867569104, -15.655823782564553, -7.8501259341093075, 2.2700234580952383e-08]


## Bestimmung der Winkel zwischen den Klötzen

Die folgende Funktion bestimmt den Steigungswinkel der Sekante durch die Punkte p1 und p2:

In [22]:
def sekantenSteigungsWinkel(p1, p2):
    m = (kettenlinie(p1)-kettenlinie(p2))/(p1-p2) #Steigung wird bestimmt
    return (np.arctan(m)/(2 * math.pi) * 360) #bestimmung des Winkels und umrechnen von Radiant in Grad

--------

Die folgende Funktion liefert eine Liste mit den Sekantensteiungswinkeln, welche jeweils durch einen Klotz laufen.<br>
Dafür wird erst der erste Steiungswinkel hinzugefügt und zu letzt der letzte (siehe PDF Besonderheiten erster und letzter Klotz). Dazwischen läuft eine for-Schleife durch die Liste der x-Werte und ruf jeweils die Funktion sekantenSteigungsWinkel(x_werte[i], x_werte[i+1]) mit dem momentanen und dem darauf folgenden x-Wert auf.

In [23]:
def sekantenSteigungsWinkelListe(x_werte):
    steigungsWinkel = []
    steigungsWinkel.append( np.arctan( (kettenlinie(x_werte[0])-(-1)*kettenlinie(x_werte[1]))/(x_werte[0]-x_werte[1]) ) / (2*math.pi)*360 ) # siehe PDF Besonderheiten erster und letzter Klotz
    for i in range(0,len(x_werte)-1): # von 0 bis len-1
        steigungsWinkel.append(sekantenSteigungsWinkel(x_werte[i], x_werte[i+1]))
    
    steigungsWinkel.append( np.arctan( (kettenlinie(x_werte[len(x_werte)-1])-(-1)*kettenlinie(x_werte[len(x_werte)-2]))/(x_werte[len(x_werte)-1]-x_werte[len(x_werte)-2]) ) / (2*math.pi)*360 ) ##
    return steigungsWinkel

-------------

Als nächstes wird die Hälfte der Winkel zwischen den Stücken bestimmt und als Liste zurück gegeben. Dafür wird die Liste der Sekantensteigungswinkel als Parameter übergeben.<br>
Der erste und letzte Wert können aus der Liste übernommen werden.(Siehe PDF wie erster/letzter Winkel bestimmt!!!!!!!!)<br>
Zwischen dem ersten und letzten Wert läuft wieder eine for-Schleife durch die Liste und bestimmt die Hälfte des Winkels zwischen den Sekanten.

In [24]:
def winkelZwischenStueckenHalbe(sekantenSteigungsWinkel, xWerte):
    listeWinkel = []
    listeWinkel.append(sekantenSteigungsWinkel[0])
    for i in range(1,len(sekantenSteigungsWinkel)-2):
        listeWinkel.append((180 - sekantenSteigungsWinkel[i+1] + sekantenSteigungsWinkel[i])/2)
    listeWinkel.append(abs(sekantenSteigungsWinkel[len(sekantenSteigungsWinkel)-1]))
    return listeWinkel

## Bestimmung der Klotzmaße

Um die Klotzmaße zu bestimmen, wird der Winkel in einem Klotz, an der Schrägen benötigt.<br>
Dafür wird die Liste mit den Winkeln zwischen den Klötzen benötigt (Übergabe als Parameter). Winkel $\alpha$ in den Klötzen wird bestimmt durch $\alpha = 90° - \gamma$, wobei $\gamma$ die Hälfte des Winkels zwischen den Klötzen ist. 

In [25]:
def winkelInHolzStueckAnKanteListe(winkelZwischenStueckenListe):
    winkelInHolzStueck = []
    for i in range(0,len(winkelZwischenStueckenListe)):
        winkelInHolzStueck.append(90 - winkelZwischenStueckenListe[i])
    return winkelInHolzStueck

----------

Zu Beginn wird die Länge der Sekanten, welche die Klötze beschreiben, bestimmt.<br>
Dies erfolgt, indem die Funktion mittels einer for-Schleife durch die Liste der x-Werte läuft und immer den aktuellen und den folgenden x-Wert nimmt. Zu diesen werden die y-Werte bestimmt und mit Hilfe des Satzes von Pythagoras wird die Länge der Sekanten bestimmt. (siehe PDF!!!!!)

In [26]:
def sekantenLaenge(xWerteListe):
    sekantenLaengen = []
    for i in range(0, len(xWerteListe)-1):
        y1 = kettenlinie(xWerteListe[i])
        y2 = kettenlinie(xWerteListe[i+1])
        sekantenLaengen.append(math.sqrt((xWerteListe[i+1]-xWerteListe[i])**2 + (y2-y1)**2))
    return sekantenLaengen

--------------

Da die Sekanten durch die Mitte der Klötze läuft, kommen durch die Schräge beim schneiden auf der Außenseite des Bogens noch Längen dazu und innerhalb fallen welche weg.(siehe PDF!!!)<br>
Die Länge, die außen dazukommt, ist die gleich, wie die, die innen weg fällt. Daher reicht einen Funktion, welche die zusätzliche Länge bestimmt.<br>
Die Längen werden bestimmt, indem die Hälfte der Breite mit dem Tangens des jeweiligen Winkels im Klotz multipliziert wird. (siehe PDF!!!!)

In [27]:
def langeSeiten(winkelInHolzStueck):
    langeSeite = []
    for i in range(0,len(winkelInHolzStueck)):
        langeSeite.append(2 * math.tan(math.radians(winkelInHolzStueck[i])))
    return langeSeite

-------------------

Da nun alle Größen bekannt sind, kann die außere Länge der Klotze bestimmt werden. Da der erste und letzte Klotz von den anderen unterscheiden, liefert die folgende Methode nur die Mittleren Längen.<br>
Es wird die jeweilige Sekantenlänge mit den beiden zusetzlichen Länge addiert. Zu beachten ist, dass die beiden zusatzlängen Längen in einem Klotz verschieden voneinander sind.

In [28]:
def laengeHolzStueckeListe(sekantenLaengenListe, seitenLaengenListe):
    laengen = []
    for i in range(0,len(sekantenLaengenListe)):#1 bis len-1
        laengen.append(sekantenLaengenListe[i] + seitenLaengenListe[i] + seitenLaengenListe[i+1])
    return laengen